## 1.17 Example: Trades Table  

In this section we construct a toy trades table to demonstrate the power of q-sql. 

￼A useful operator for constructing lists of test data is (?), which generates pseudo-random data. We can generate 10 numbers randomly selected, with replacement, from the first 20 integers starting at 0 (i.e., not including 20). 

In [1]:
10?20 / ymmv 

12 8 10 1 9 11 5 6 1 5


In [2]:
10?20 = 10?20

0000000000b


We can similarly generate 10 random floats between 0.0 and 100.0 (not including 100.0). 

In [3]:
10?100.0

19.46509 9.059026 62.03014 93.26316 27.47066 5.752516 25.60658 23.10108 8.724..


In [4]:
/We can make 10 random selections from the items in a list 
10?(`one;`two)
10?`one`1

`two`one`two`two`two`two`one`one`one`two


`one`one`one`one`one`1`one`1`one`1



Now to our trades table. Since a table is a collection of columns, we first build the columns. We apologize for using excessively short names so that things fit easily on the printed page. 
First we construct a list of 1,000,000 random dates in the month of January 2015. 

In [5]:
dts:2015.01.01+1000000?31

In [6]:
dts

2015.01.28 2015.01.19 2015.01.16 2015.01.27 2015.01.09 2015.01.30 2015.01.14 ..


In [7]:
tms:1000000?24:00:00.000000000

In [8]:
/Next a list of 1,000,000 tickers chosen from AAPL, GOOG and IBM. It is customary to make these lower case symbols. 

In [9]:
syms:1000000?`aapl`goog`ibm

In [10]:
/Next a list of 1,000,000 volumes given as positive lots of 10
vols:10*1+1000000?1000

As an initial cut, we construct a list of 1,000,000 prices in cents uniformly distributed within 10% of 100.0. We will adjust this later. 

In [11]:
pxs:90.0+(1000000?2001)%100

In [12]:
/Now collect these into a table and inspect the first 5 records. Remember, a table is a list of records so (#) applies.

In [13]:
trades:([] dt:dts; tm:tms; sym:syms; vol:vols; px:pxs) 

In [14]:
5#trades

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.28 0D10:54:28.956224620 goog 3650 92.38 
2015.01.19 0D19:56:35.270023494 aapl 9470 96.03 
2015.01.16 0D03:10:49.227076023 aapl 9350 99.66 
2015.01.27 0D03:10:07.354279607 aapl 7990 108.14
2015.01.09 0D16:28:56.131688207 goog 2710 104.05


The first thing you observe in your console display is that the trades are not in temporal order. We fix this by sorting on time within date using (xasc). 

In [15]:
trades:`dt`tm xasc trades 

In [16]:
5#trades

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.01 0D00:00:07.004971057 aapl 570  108.14
2015.01.01 0D00:00:09.415197372 goog 9030 104.89
2015.01.01 0D00:00:13.514088839 ibm  6180 93.07 
2015.01.01 0D00:00:14.056894183 ibm  8870 90.76 
2015.01.01 0D00:00:15.205892175 aapl 5480 103.47


Now we adjust the prices. At the time of this writing (Sep 2015) AAPL was trading around 100, so we leave it alone. But we adjust GOOG and IBM to their approximate trading ranges by scaling. 

In [17]:
trades:update px:6*px from trades where sym=`goog

In [18]:
5#trades

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.01 0D00:00:07.004971057 aapl 570  108.14
2015.01.01 0D00:00:09.415197372 goog 9030 629.34
2015.01.01 0D00:00:13.514088839 ibm  6180 93.07 
2015.01.01 0D00:00:14.056894183 ibm  8870 90.76 
2015.01.01 0D00:00:15.205892175 aapl 5480 103.47


In [19]:
trades:update px:2*px from trades where sym=`ibm

In [20]:
5#trades 

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.01 0D00:00:07.004971057 aapl 570  108.14
2015.01.01 0D00:00:09.415197372 goog 9030 629.34
2015.01.01 0D00:00:13.514088839 ibm  6180 186.14
2015.01.01 0D00:00:14.056894183 ibm  8870 181.52
2015.01.01 0D00:00:15.205892175 aapl 5480 103.47


This looks a bit more like real trades. Let’s perform some basic queries as sanity checks. Given that both price and volume are uniformly distributed, we expect their averages to approximate the mean. Using the built-in average function (avg) we see that they do. 

In [21]:
select avg px, avg vol by sym from trades 

sym | px       vol     
----| -----------------
aapl| 100.0127 5001.963
goog| 600.0906 5001.169
ibm | 199.9685 5016.99 


Similarly, we expect the minimum and maximum price for each symbol to be the endpoints of the uniform range.

In [22]:
select min px, max px by sym from trades 

sym | px  px1
----| -------
aapl| 90  110
goog| 540 660
ibm | 180 220


Our first non-trivial query computes the 100 millisecond bucketed volume-weighted average price (VWAP). This uses the built-in dyadic function (xbar). The left operand of (xbar) is an interval width and the right operand is a list of numeric values. The effect of (xbar) is to shove each input to the left-hand end point of the interval of specified width in which it falls. For example

In [23]:
5 xbar til 15

0 0 0 0 0 5 5 5 5 5 10 10 10 10 10


In [24]:
10 xbar til 30

0 0 0 0 0 0 0 0 0 0 10 10 10 10 10 10 10 10 10 10 20 20 20 20 20 20 20 20 20 20


This is useful for grouping since it effectively buckets all the values within each interval to the left end-point of that interval. Recalling that a timespan is actually an integral count of nanoseconds since midnight, to compute 100 millisecond buckets we will use (‘’’xbar’’’) with an interval of 100,000,000. 

We also require (‘’’wavg’’’), a dyadic function that computes the average of the numeric values in its right operand weighted by the values of its left operand. 

In [25]:
1 2 3 wavg 50 60 70

63.33333


In [26]:
1 2 9 wavg 50 60 70

66.66667


In [27]:
10 2 3 wavg 50 60 70

55.33333


In [28]:
5#(select vwap:vol wavg px by sym,bkt:100000000 xbar tm from trades)

sym  bkt                 | vwap    
-------------------------| --------
aapl 0D00:00:00.000000000| 105.47  
aapl 0D00:00:00.300000000| 109.67  
aapl 0D00:00:00.400000000| 98.37   
aapl 0D00:00:00.600000000| 97.58849
aapl 0D00:00:01.100000000| 95.0323 


That’s all there is to it! 

Our final query involves the maximum profit (or analogously, maximum drawdown) realizable over the trading period. To understand the concept, image that you have a DeLorean with flux capacitor and are able to travel into the future and record historical trade results. Upon returning to the present, you are given $1,000,000 to invest with the stipulation that you can make one buy and one sell for AAPL and you are not allowed to short the stock. As a good capitalist your goal is to maximize your profit. 

Restating the problem, we wish to determine the optimum time to buy and sell for the largest (positive) difference in price, where the buy precedes the sell. We state the solution as a q koan, which you should contemplate until enlightenmen

In [29]:
q)select max px-mins px from trades where sym=`aapl

px
--
20


In [30]:
select dt,tms from trades where px =( maxp: (select max px from trades))

[0;31mmaxp[0m: [0;31mmaxp[0m

In [33]:
select max px from trades where sym = `goog

px 
---
660
